In [ ]:
from grid.refine import *

df = observed_rates()

keys = {}
for i, r in df.iterrows():
    ks = {}
    for color in ['black', 'white']:
        name = r[f'{color}_name']
        key = f'{color}_key'
        if name.startswith('mohex'):
            ks[key] = f'{r.boardsize}-{name}'
        elif name == 'latest':
            ks[key] = f'{r.boardsize}B{r.width}W{r.depth}D+S'
        else:
            idx = name.split('.')[1] 
            ks[key] = f'{r.boardsize}B{r.width}W{r.depth}D{idx}S'
    keys[i] = ks
df = pd.concat([df, pd.DataFrame.from_dict(keys, orient='index')], 1)

keyed = df[['black_key', 'white_key', 'black_wins', 'white_wins']]
for b in [3, 5, 7, 9]:
    aux = database.pandas(f'mohex-{b}').reset_index()
    aux['black_key'] = f'{b}-' + aux.black_name
    aux['white_key'] = f'{b}-' + aux.white_name
    keyed = pd.concat([keyed, aux.reindex(columns=df.columns)])

black_wins = pd.pivot_table(keyed, 'black_wins', 'black_key', 'white_key', aggfunc='sum').fillna(0)
white_wins = pd.pivot_table(keyed, 'white_wins', 'black_key', 'white_key', aggfunc='sum').fillna(0)

wins = black_wins + white_wins.T
games = black_wins + white_wins + black_wins.T + white_wins.T

targets = set(df.black_key.values) | set(df.white_key.values)
targets = [t for t in targets if 'mohex' not in t]

In [ ]:
with aljpy.parallel(solve) as p:
    results = p.wait({t: p(t, games, wins) for t in targets})

In [ ]:
results = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
aux = (df
       .groupby('black_key').first()
       .loc[lambda df: ~df.index.str.contains('mohex')]
       .loc[:, ['run', 'boardsize', 'width', 'depth', 'black_name']])

trans = {}
for r, g in tqdm(aux.groupby('run')):
    times = {f'snapshot.{idx}': pd.to_datetime(info['_created']) for idx, info in storage.snapshots(r).items()}
    
    samples = pd.DataFrame(stats.array(r, 'count.samples')).set_index('_time').tz_localize('UTC').cumsum()['total']
    samps = {}
    for s, t in times.items():
        samps[s] = samples[:t].iloc[-1]
    # This is approximate and I hate it
    samps['latest'] = samples.iloc[-1]
    
    s = pd.Series(samps)[g.black_name.values]
    s.index = g.index
    trans[r] = s
trans = pd.concat(trans).reset_index()
trans.columns = ['run', 'black_key', 'samples']

aux = pd.concat([aux, trans.set_index('black_key')['samples']], 1)

In [ ]:
joint = pd.concat([aux, solns], 1)
joint['g'] = joint.index.str[:-2]
joint['flops'] = plots.flops(joint)
joint['params'] = plots.params(joint)

In [ ]:
(ggplot(data=joint[joint['black_name'] != 'snapshot.0'])
    + geom_point(aes(x='flops', y='μ', group='g', color='params'))
    + scale_x_continuous(trans='log10')
    + scale_color_continuous(trans='log10')
    + facet_wrap('boardsize', labeller='both')
    + labs(title='incomplete refine results: i need to isolate the frontier and nail that down')
    + plots.mpl_theme(12, 8)
    + plots.poster_sizes())

In [ ]:
from 